INSTALL PACKAGES

In [ ]:
#install required packages
#run this cell only once if not already installed
#the '!' is used to run shell commands in jupyter/collab
! pip install langchain-groq langchain_community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu
#langchain-community : community-contirbuted integrations for langchain like faiss,hugging face,etc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


IMPORT LIBRARIES

In [ ]:
#load environment variables from a '.env' file  into the
#useful for storing api keys and configuartion settings
from dotenv import load_dotenv
#PDF reading library-allows you to open and extract text from pdf files
from PyPDF2 import PdfReader
#load_dotenv()
from langchain.text_splitter import CharacterTextSplitter
#embedding model wrapper for hugging face models (like snetence transformers)
from langchain_huggingface import HuggingFaceEmbeddings
#wrapper for
from langchain_groq import ChatGroq
from  langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from huggingface_hub import login
#used to access environment variables,file paths,etc
import os
from langchain.prompts import PromptTemplate


LOAD API KEYS

In [ ]:
#load environment variables and login to hugging face
load_dotenv()
login(os.getenv("HUGGINGFACEHUB_API_hf_tTPSNBNEEBzaCuwxNumoXXFoPgrPDRluUZ"))

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving research paper_academic.pdf to research paper_academic.pdf


In [ ]:
pdf_paths =['/content/research paper_academic.pdf']

In [ ]:
def get_pdf_text(pdf_docs):
  text =""
  for pdf in pdf_docs:
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
       page_text =page.extract_text()
       if page_text:
          text+=page_text
    return text #return the combined text from all PDF pages
#call the function with the list of uploaded pdf paths
raw_text = get_pdf_text(pdf_paths)
#print the lenght of the extracted text to verify it worked
print("Extracted text length:", len(raw_text))

Extracted text length: 53091


In [ ]:
def get_text_chunks(text):
  text_splitter = CharacterTextSplitter(
      separator='\n',
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )
  chunks=text_splitter.split_text(raw_text)
  return chunks
text_chunks = get_text_chunks(raw_text)
print("Number of text chunks:",len(text_chunks))

Number of text chunks: 68


In [ ]:
def get_vectorstore(text_chunks):
  embeddings =HuggingFaceEmbeddings(
      model_name="hkunlp/instructor-xl",
      model_kwargs={"device":"cuda"}
  )
  vectorstore =FAISS.from_texts(
      texts=text_chunks, # list of text chunks from the pdf
      embedding=embeddings #embedding model is used to convert text to vectors
  )
  return vectorstore
#create the vector store from previously created text chunks
vectorstore =get_vectorstore(text_chunks)
print("Vectorstore created")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Vectorstore created


In [ ]:
def get_conversation_chain(vectorstore):
  llm = ChatGroq(
      model_name="meta-llama/llama-4-scout-17b-16e-instruct",
      temperature=0.7,
      request_timeout=30
  )
  memory=ConversationBufferMemory(
      memory_key='chat_history',
      return_messages=True
  )

In [ ]:
db=vectorstore

In [ ]:
retriever=db.as_retriever

In [ ]:
template = """
You are an expert science communicator. Based on the content of the research paper below, write a blog-style explainer suitable for a non-expert audience. Follow this format:

Title: (Make it catchy and easy to understand)

1. The Problem
- What problem does the paper address?
- Why is it important?

2. The Approach
- What method or idea is proposed?
- How does it work in simple terms?

3. Key Takeaways
- What are the key findings?
- How can this be useful?

Guidelines:
- Assume the reader has no technical background
- Do not use formulas or code
- Use a friendly, engaging, and educational tone
- Keep it under 800 words

Context: {context}
Research Paper Text: {question}
Generate a blog-style explainer based on the research above:
"""
prompt=PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [ ]:
! pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
import os
os.environ["GROQ_API_KEY"]="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"

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
llm= ChatGroq(
    temperature=0,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)

In [ ]:
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query = "Write a blog-style summary of this research paper for non experts readers."
response = qa_chain.run(query)
print("✅ Blog :\n", response)

/tmp/ipython-input-25-1197253732.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


✅ Blog :
 **Can AI Accurately Judge Academic Research?**

**The Problem**

Imagine you're an academic trying to get your research published. You submit your paper to a journal, and it's reviewed by an expert in the field. But what if that expert was not a human, but an artificial intelligence (AI) system? Would it be able to accurately assess the quality of your research? This is the problem that a recent study aimed to address.

The study's goal was to see if AI systems, like ChatGPT, can effectively evaluate academic research papers. This is important because AI is increasingly being used to help with tasks like peer review, which can be time-consuming and labor-intensive for humans. However, if AI systems can't accurately assess research, it could have significant consequences for the academic community.

**The Approach**

The researchers tested ChatGPT's ability to evaluate academic papers using a dataset of research papers from various fields. They provided ChatGPT with the titles